In [ ]:
import os
os.environ['HF_TOKEN']="***"

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitsAndBytesConfig
import torch, json, gc, time
import pandas as pd
import numpy as np
# import google.generativeai as genai
set_seed(1234)

models = {"Llama-3.1-8B":"meta-llama/Llama-3.1-8B-Instruct", "Ministral-8B-Instruct": "mistralai/Ministral-8B-Instruct-2410"}

# Define the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

In [4]:
def generate_response(system_message, user_message, tokenizer, model):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)  # 양자화된 모델 사용

    # attention_mask 생성
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    
    attention_mask = (input_ids != pad_token_id).long()

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.01,
        top_p=0.3
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)


In [5]:
data = pd.read_csv('/home/user/Desktop/NewsToCartoon/gold_cnn_dailymail.csv')

In [7]:
prompt = """
    Find the 5W1H that is central to the article (input).

    Output format (Write in English): 
    Main Event:
    Who:
    Where:
    When:
    What:
    Why:
    How:
    """

In [8]:
main_event_list = []
who_list = []
where_list = []
when_list = []
what_list = []
why_list = []
how_list = []

In [9]:
torch.cuda.empty_cache()
gc.collect()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [10]:
loaded_models = {}
for model_name, model_id in models.items():
    print(f"Loading {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="cuda"
    )
    loaded_models[model_name] = (tokenizer, model)

Loading Llama-3.1-8B...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading Ministral-8B-Instruct...


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
from dotenv import load_dotenv
from openai import OpenAI
import time

# API KEY 정보로드
load_dotenv()

True

In [12]:
print(data.columns)

Index(['article', 'highlight'], dtype='object')


In [13]:
import os
import json

# 결과를 저장할 폴더 생성
output_folder = "output_results"
os.makedirs(output_folder, exist_ok=True)

# 데이터를 처리
for idx, row in data.iterrows():
    input_text = row['article']
    result_data = {"input_text": input_text, "results": {}}

    # OpenAI API 호출
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": [{"type": "text", "text": prompt}]},
            {"role": "user", "content": [{"type": "text", "text": input_text}]},
        ],
        max_tokens=512,
        temperature=0,
        top_p=0.2,
    )
    openai_output = response.choices[0].message.content
    print("OpenAI output:", openai_output)

    # OpenAI API 결과 저장
    result_data["results"]["GPT-4o"] = openai_output
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": [{"type": "text", "text": prompt}]},
            {"role": "user", "content": [{"type": "text", "text": input_text}]},
        ],
        max_tokens=512,
        temperature=0,
        top_p=0.2,
    )
    mini_output = response.choices[0].message.content
    print("OpenAI output:", mini_output)

    # OpenAI API 결과 저장
    result_data["results"]["GPT-4o-mini"] = mini_output

    # 모델별로 결과 생성
    for model_name, (tokenizer, model) in loaded_models.items():
        print(f"{model_name} start!")
        
        # generate_response 함수 호출
        summary_text = generate_response(
            system_message=prompt,
            user_message=input_text,
            tokenizer=tokenizer,
            model=model
        )
        print(f"{model_name} output:", summary_text)

        # 모델 결과 저장
        result_data["results"][model_name] = summary_text

    # JSON 파일로 저장
    output_file_path = os.path.join(output_folder, f"result_{idx + 1}.json")
    with open(output_file_path, "w", encoding="utf-8") as json_file:
        json.dump(result_data, json_file, indent=4, ensure_ascii=False)

    print(f"Results saved to {output_file_path}")

# 모델 자원 해제 (메모리 최적화를 위해)
for model_name, (_, model) in loaded_models.items():
    del model
torch.cuda.empty_cache()


OpenAI output: Main Event: Daniel Radcliffe gains access to his fortune as he turns 18.

Who: Daniel Radcliffe

Where: London, England

When: As he turns 18 on Monday

What: Daniel Radcliffe gains access to a reported £20 million fortune.

Why: He turns 18, which allows him to access his earnings from the Harry Potter films held in a trust fund.

How: By reaching the age of 18, he legally gains control over his trust fund.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OpenAI output: Main Event: Daniel Radcliffe turns 18 and gains access to his £20 million fortune.  
Who: Daniel Radcliffe  
Where: London, England  
When: On his 18th birthday, which is a Monday  
What: He discusses his plans for his newfound wealth and adulthood.  
Why: To express his intentions regarding his fortune and to counteract the stereotype of child stars going off the rails.  
How: Through interviews and public statements about his spending habits and future plans.
Llama-3.1-8B start!


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Llama-3.1-8B output: Main Event: Daniel Radcliffe turns 18 and gains access to his reported £20 million fortune.

1. Who:
   - Daniel Radcliffe: The Harry Potter star who turns 18 and gains access to his reported £20 million fortune.

2. Where:
   - London, England: The location where Daniel Radcliffe lives and where he will be able to access his fortune.

3. When:
   - Monday: The day Daniel Radcliffe turns 18 and gains access to his reported £20 million fortune.

4. What:
   - Gaining access to a reported £20 million fortune: Daniel Radcliffe will be able to access his earnings from the first five Potter films, which have been held in a trust fund.

5. Why:
   - To mark his 18th birthday: Daniel Radcliffe wants to celebrate his milestone birthday and have some control over his finances.

6. How:
   - By turning 18: Daniel Radcliffe will be able to access his reported £20 million fortune as he turns 18, allowing him to make his own financial decisions.
Ministral-8B-Instruct start!
Min

...

Llama-3.1-8B output: Main Event: 
South African fast bowler Dale Steyn took a career-best five for 34 as the Proteas took a tight grip on the first test against New Zealand in Johannesburg.

Who:
- Dale Steyn (South African fast bowler)
- Hashim Amla (South African batsman)
- Jacques Kallis (South African batsman)
- Stephen Fleming (Former New Zealand captain)
- Ross Taylor (New Zealand batsman)
- Shane Bond (New Zealand nightwatchman)
- AB de Villiers (South African batsman)
- John Bracewell (New Zealand coach)
- Graeme Smith (South African captain)
- Herschelle Gibbs (South African opener)
- Brendon McCullum (New Zealand player)

Where:
- Johannesburg, South Africa

When:
- The article does not specify the exact date of the event, but it mentions that it was two seasons ago when Dale Steyn took a five-wicket haul against the same opponents at Centurion.

What:
- Dale Steyn took a career-best five for 34 as the Proteas took a tight grip on the first test against New Zealand in Johanne

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OpenAI output: Main Event: Five great ways to enjoy summer  
Who: Caroline Calvin, Lynn Havsall, Benjamin Guss, Penny Waugh, Meghan Cleary  
Where: Various locations including Bar Harbor, Maine, Del Mar, California  
When: Summer  
What: Tips for enjoying summer activities such as cutting jeans into shorts, catching fireflies, running on the beach, getting in and out of a hammock, and tying espadrilles  
Why: To provide readers with enjoyable and practical ways to make the most of their summer  
How: Through expert advice and step-by-step instructions for each activity
Llama-3.1-8B start!


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Llama-3.1-8B output: Main Event: Enjoying Summer

1. Who:
   - Caroline Calvin, creative director of Levi's
   - Lynn Havsall, director of programs at the George B. Dorr Museum of Natural History
   - Benjamin Guss, lifeguard from Del Mar, California
   - Penny Waugh, a buyer for hammocks.com
   - Meghan Cleary, author of "The Perfect Fit: What Your Shoes Say About You"

2. Where:
   - Various locations, including beaches and hammocks

3. When:
   - Summer evenings

4. What:
   - Enjoying summer by various activities such as:
     - Lazing in a hammock
     - Cutting jeans into shorts
     - Catching fireflies
     - Running on the beach
     - Getting in and out of a hammock
     - Tying espadrilles

5. Why:
   - To spend a relaxing summer evening
   - To get in shape by running on the beach
   - To have fun catching fireflies
   - To look stylish by tying espadrilles correctly

6. How:
   - To cut jeans into shorts: mark the desired length, fold the leg, iron the fold, and cut just u

In [24]:
# 열 순서를 재정렬
columns_order = []
categories = ["Who", "Where", "When", "What", "Why", "How"]
for category in categories:
    for model_name in ["OpenAI"] + list(loaded_models.keys()):
        column_name = f"{category}_{model_name}"
        if column_name in data.columns:
            columns_order.append(column_name)

# 데이터프레임 열 순서 변경
data = data[columns_order]

# 결과를 새로운 CSV 파일로 저장
data.to_csv('output_file.csv', index=False)

# 모델 자원 해제 (메모리 최적화를 위해)
for model_name, (_, model) in loaded_models.items():
    del model
torch.cuda.empty_cache()